In [1]:
import gc
import glob
import os
from os import path
import json
import matplotlib.pyplot as plt
import seaborn as sns
import pprint
import warnings
from joblib import Parallel, delayed
from tqdm import tqdm, tqdm_notebook

from collections import Counter, defaultdict
from functools import partial
from math import sqrt
import time
from functools import reduce
from numba import cuda
import math
import random

import numpy as np
import pandas as pd
import scipy as sp
import xgboost as xgb
import lightgbm as lgb

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import cohen_kappa_score, mean_squared_error
from sklearn.metrics import confusion_matrix as sk_cmatrix
from sklearn.model_selection import StratifiedKFold, GroupKFold

from keras.applications.densenet import preprocess_input, DenseNet121
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Input, Lambda, AveragePooling1D, \
        concatenate, BatchNormalization, Activation, Dropout, Embedding, Reshape
from keras.callbacks import LearningRateScheduler
from keras.optimizers import *
from keras.initializers import glorot_normal
import keras.backend as K

%matplotlib inline

np.random.seed(seed=1)
warnings.filterwarnings('ignore')

split_char = '/'

Using TensorFlow backend.


In [2]:
train = pd.read_csv('../input/train.csv')
train0 = train[ train['target']==0 ].copy()
train1 = train[ train['target']==1 ].copy()
train.sample(5)

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
49673,train_49673,0,11.5208,0.0345,13.6967,8.7980,11.4984,-14.4183,5.2356,14.6201,...,-2.5931,6.8155,4.0026,5.7603,16.7222,0.5767,-3.1717,8.9964,19.5975,-16.0881
171551,train_171551,0,10.6327,-3.0837,7.3320,8.9173,9.8485,-10.0986,5.4894,11.2602,...,7.2326,5.0710,3.4117,4.1256,19.3753,-2.2722,-2.5486,6.3672,16.0392,-0.8213
5506,train_5506,0,13.4346,-4.7138,10.2488,7.2394,11.5154,-10.7816,4.4755,16.3473,...,1.5606,14.4879,1.8716,6.0999,21.1498,3.1641,-3.9446,9.4770,17.1406,-18.9208
38370,train_38370,0,4.8650,4.3326,13.9920,6.9034,12.1384,-1.6310,5.2511,14.0651,...,-5.9529,5.7586,3.0283,-2.4441,16.7881,-0.2528,-5.1544,8.4277,12.8527,1.1520
36930,train_36930,0,9.8466,-2.0145,10.6657,4.9808,10.0866,-7.7280,4.6048,16.5240,...,-0.5729,9.0527,1.2550,-4.4527,16.0889,0.8156,4.3412,9.0945,13.4120,8.3931


In [3]:
train.shape

(200000, 202)

In [4]:
# CALCULATE MEANS AND STANDARD DEVIATIONS
s = [0]*200
m = [0]*200
for i in range(200):
    s[i] = np.std(train['var_'+str(i)])
    m[i] = np.mean(train['var_'+str(i)])
    
# CALCULATE PROB(TARGET=1 | X)
def getp(i,x):
    c = 3 #smoothing factor
    a = len( train1[ (train1['var_'+str(i)]>x-s[i]/c)&(train1['var_'+str(i)]<x+s[i]/c) ] ) 
    b = len( train0[ (train0['var_'+str(i)]>x-s[i]/c)&(train0['var_'+str(i)]<x+s[i]/c) ] )
    if a+b<500: return 0.1 #smoothing factor
    # RETURN PROBABILITY
    return a / (a+b)
    # ALTERNATIVELY RETURN ODDS
    # return a / b
    
# SMOOTH A DISCRETE FUNCTION
def smooth(x,st=1):
    for j in range(st):
        x2 = np.ones(len(x)) * 0.1
        for i in range(len(x)-2):
            x2[i+1] = 0.25*x[i]+0.5*x[i+1]+0.25*x[i+2]
        x = x2.copy()
    return x

In [5]:
def getp2(i,x):
    z = (x-m[i])/s[i]
    ss = (rmax-rmin)/(res-1)
    if res%2==0: idx = min( (res+1)//2 + z//ss, res-1)
    else: idx = min( (res+1)//2 + (z-ss/2)//ss, res-1)
    idx = max(idx,0)
    return pr[i,int(idx)]

In [6]:

# DATA HAS Z-SCORE RANGE OF -4.5 TO 4.5
rmin=-5; rmax=5; 
# CALCULATE PROBABILITIES FOR 501 BINS
res=501
# STORE PROBABILITIES IN PR
pr = 0.1 * np.ones((200,res))
pr2 = pr.copy()
xr = np.zeros((200,res))
xr2 = xr.copy()
ct2 = 0
for j in range(50):
    for v in range(4):
        ct = 0
        # CALCULATE PROBABILITY FUNCTION FOR VAR
        for i in np.linspace(rmin,rmax,res):
            pr[v+4*j,ct] = getp(v+4*j,m[v+4*j]+i*s[v+4*j])
            xr[v+4*j,ct] = m[v+4*j]+i*s[v+4*j]
            xr2[v+4*j,ct] = i
            ct += 1

KeyboardInterrupt: 

In [ ]:
np.min(pr)

In [ ]:
from sklearn.metrics import roc_auc_score
print('Calculating 200000 predictions and displaying a few examples...')
pred = [0]*200000; ct = 0
for r in train.index:
    p = 0.1
    for i in range(200):
        p *= 10*getp2(i,train.iloc[r,2+i])
    if ct%25000==0: print('train',r,'has target =',train.iloc[r,1],'and prediction =',p)
    pred[ct]=p; ct += 1
print('###############')
print('Validation AUC =',roc_auc_score(train['target'], pred))